<a href="https://colab.research.google.com/github/Gricay-vasily/project_9_DCGAN/blob/main/generator_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from PIL import Image

# Завантаження та підготовка датасету CIFAR-10
def load_cifar10():
    (x_train, y_train), (_, _) = cifar10.load_data()
    x_train = (x_train.astype('float32') - 127.5) / 127.5  # нормалізація до [-1, 1]
    return x_train

# Створення генератора
def build_generator(nodes=4, input_dim=100, alpha=0.2):
    model = tf.keras.Sequential()

    n_f_nodes = 256 * nodes * nodes
    model.add(layers.Dense(n_f_nodes, input_dim=input_dim))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Reshape((nodes, nodes, 256)))

    model.add(layers.Conv2DTranspose(128, (nodes, nodes), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(128, (nodes, nodes), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(128, (nodes, nodes), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha))

    model.add(layers.Conv2D(3, (3, 3), activation='tanh', padding='same'))

    return model

# Створення дискримінатора
def build_discriminator(input_shape=(32, 32, 3), alpha=0.2):
    model = tf.keras.Sequential()

    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=input_shape))
    model.add(layers.LeakyReLU(alpha))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# Побудова та компіляція GAN
def build_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
    discriminator.trainable = False

    gan_input = layers.Input(shape=(100,))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)

    gan = models.Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

    return gan

# Навчання GAN
def train_gan(generator, discriminator, gan, data, epochs=10000, batch_size=64):
    for epoch in range(epochs):
        # Вибірка реальних зображень
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_images = data[idx]

        # Генерація випадкових шумів та фейкових зображень
        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_images = generator.predict(noise)

        # Створення міток для реальних та фейкових зображень
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        # Навчання дискримінатора
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Генерація шуму для навчання генератора через GAN
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_labels = np.ones((batch_size, 1))

        # Навчання генератора через GAN
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Вивід прогресу навчання
        if epoch % 1000 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}%] [G loss: {g_loss}]")

# Завантаження датасету
data = load_cifar10()

# Створення генератора та дискримінатора
generator = build_generator()
discriminator = build_discriminator()

# Побудова та компіляція GAN
gan = build_gan(generator, discriminator)

# Навчання GAN
train_gan(generator, discriminator, gan, data, epochs=10000, batch_size=64)

# Функція для передобробки зразка зображення
def preprocess_image(image_path, target_size=(32, 32)):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image_array = np.array(image) / 127.5 - 1  # нормалізація до [-1, 1]
    image_array = np.expand_dims(image_array, axis=0)  # додавання batch dimension
    return image_array

# Функція для генерації зображення
def generate_image(generator, sample_image):
    # Підготовка латентного простору (шуму)
    noise = np.random.normal(0, 1, (1, 100))  # можна покращити шляхом використання особливостей sample_image

    # Генерація зображення
    generated_image = generator.predict(noise)
    generated_image = (generated_image * 127.5 + 127.5).astype(np.uint8)  # зворотна нормалізація до [0, 255]
    return generated_image

# Завантаження та передобробка зображення
sample_image_path = 'cat2.jpg'  # замініть на шлях до вашого зразка
sample_image = preprocess_image(sample_image_path)

# Генерація зображення
generated_image = generate_image(generator, sample_image)

# Візуалізація згенерованого зображення
plt.imshow(generated_image[0])
plt.axis('off')
plt.show()


2/2 [==============================] - 1s 203ms/step
0 [D loss: 0.7017967104911804, acc.: 15.625%] [G loss: 0.6034762859344482]
2/2 [==============================] - 0s 201ms/step
